<center><h1>Project XII</h1><h2>Machine Learning in Business</h2></center>

<b><u>Description:</u></b>

We work at OilyGiant mining company. Our task is to find the best place for a new well. We were provided with 3 datasets of 3 different regions: df_1, df_2, df_3.


<b><u>Our plan:</u></b>

- Collect the oil well parameters in the selected region: oil quality and volume of reserves;

- Build a model for predicting the volume of reserves in the new wells;

- Pick the oil wells with the highest estimated values;

- Pick the region with the highest total profit for the selected oil wells.

<hr><div id="Step I"><h2>Step I - Open data and general info</h2><br><i>Loading all the libraries: pandas,numpy,matplotlib,etc</i></div>


In [1]:
# import pandas and numpy for data preprocessing and manipulation
import pandas as pd
import numpy as np
import re

# seaborn for visualization
import seaborn as sns
sns.set_style("darkgrid")

# matplotlib for visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# import sklearn modules
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler as ss
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
# from sklearn.dummy import DummyClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# import warnings and display html
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display_html
from itertools import chain,cycle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
pd.set_option('display.max_rows', None)

print('Project libraries has been successfully imported!')

Project libraries has been successfully imported!


<hr><i>Loading Dataset</i>


In [2]:
# read the data

try:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')
except:
    df_0 = pd.read_csv(r'C:\Users\vlady\Downloads\geo_data_0.csv')
    
print('Data df_0 has been read correctly!')

try:
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')
except:
    df_1 = pd.read_csv(r'C:\Users\vlady\Downloads\geo_data_1.csv')
    
print('Data df_1 has been read correctly!')

try:
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df_2 = pd.read_csv(r'C:\Users\vlady\Downloads\geo_data_2.csv')
    
print('Data df_2 has been read correctly!')

Data df_0 has been read correctly!
Data df_1 has been read correctly!
Data df_2 has been read correctly!



**Features**

 
 - `f0` - features of unknown parameter 0
 
 
 - `f1` - features of unknown parameter 1
 
 
 - `f2` - features of unknown parameter 2
 
 
 - `product` - volume of reserves in the oil well (thousand barrels).
 
 
 - `id` - unique oil well identifier



### Data Checking functions:

In [3]:

# function to determine if columns in file have null values        
def get_percent_of_na(df, num):
    count = 0
    df = df.copy()
    s = (df.isna().sum() / df.shape[0])
    for column, percent in zip(s.index, s.values):
        num_of_nulls = df[column].isna().sum()
        if num_of_nulls == 0:
            continue
        else:
            count += 1
        print('{} has {} nulls, which is {:.{}%} percent of Nulls'.format(column, num_of_nulls, percent, num))
    if count != 0:
        print("\033[1m" + 'There are {} columns with NA.'.format(count) + "\033[0m")
    else:
        print()
        print("\033[1m" + 'There are no columns with NA.' + "\033[0m")       
        
# function to display general information about the dataset
def general_info(df):
    print("\033[1m" + "\033[0m")
    display(pd.concat([df.dtypes, df.count(),df.isna().sum(),df.isna().sum()/len(df)], keys=['type','count','na','na%'],
                      axis=1))
    print()
    print("\033[1m" + 'Sample:')  
    display(df.sample(5))
    print()
    print("\033[1m" + 'Info:')
    print()
    display(df.info())
    print()
    print("\033[1m" + 'Describe:')
    print()
    display(df.describe())
    print()
    print("\033[1m" + 'Nulls in the columns:') 
    display(df.isna().sum())
    print()
    print("\033[1m" + 'Zeros in the columns:') 
    print()
    print("\033[1m" + 'Shape:', df.shape)
    print()
    print("Unique wells in the dataset :", df.id.nunique())
    print()
    print('Duplicated:',"\033[1m" + 'We have {} duplicated rows\n'.format(df.duplicated().sum()) + "\033[0m")
    print()
    print("\033[1m" + 'Dtypes:')  
    display(df.dtypes)
    print()


In [4]:
#print our info data
print('Inspection of df_0 dataset:')
general_info(df_0)

Inspection of df_0 dataset:



,type,count,na,na%
id,object,100000,0,0.0
f0,float64,100000,0,0.0
f1,float64,100000,0,0.0
f2,float64,100000,0,0.0
product,float64,100000,0,0.0



Sample:


,id,f0,f1,f2,product
92158,Dielf,-0.246867,1.139938,3.787254,124.944251
97973,HsAzR,1.665654,-0.215909,6.418615,183.209590
24948,y1i8e,2.120696,0.290109,2.529260,104.558008
61438,iOAHs,0.139714,1.156531,4.834322,36.894584
87303,1pLfe,-0.477732,0.967057,3.216607,54.500438



Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Nulls in the columns:


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Zeros in the columns:

Shape: (100000, 5)

Unique wells in the dataset : 99990

Duplicated: We have 0 duplicated rows


Dtypes:


id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

In [5]:
# Checking the number of duplicated ids
df_0.id.duplicated().sum()


10

<hr><h2>Fixing df_0</h2>

- Our analysis shows that there are **10 instances of not unique well ids**. The original dataset contains 100K instances, yet only 99,990 are unique. We will average the parameters of these non uniq wells and drop the duplicated ids.    



In [6]:
df_0 = df_0.groupby('id').mean().reset_index()

In [7]:
# Checking that we have no duplicates in id column
df_0.id.duplicated().sum()

0

In [8]:
# Checking that the lenght of the dataset is equal to the unique instances of the dataset. 
df_0.id.nunique() == len(df_0)

True

## Checking DF_1

In [9]:
general_info(df_1)

,type,count,na,na%
id,object,100000,0,0.0
f0,float64,100000,0,0.0
f1,float64,100000,0,0.0
f2,float64,100000,0,0.0
product,float64,100000,0,0.0



Sample:


,id,f0,f1,f2,product
91368,WYKPd,-5.708452,-5.924074,3.997003,110.992147
49825,WsEZg,9.321891,-14.028430,2.999607,80.859783
20118,i5tQq,-13.322438,-6.834484,2.002320,57.085625
19345,jCkRm,6.012912,-4.072312,2.994306,80.859783
17936,4y0oR,-4.785439,0.620117,0.002984,3.179103



Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

...

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



Nulls in the columns:


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Zeros in the columns:

Shape: (100000, 5)

Unique wells in the dataset : 99996

Duplicated: We have 0 duplicated rows


Dtypes:


id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

## Fixing df_1:

df_1 dataset contains extremly large proportion of zero values in the product column (8235 instances). For now we decided to leave it as is, but it is something that we should keep in mind (df_0, for example had only 1 instance of product ==0 instance).

We also came across duplicates in the id column of df_1. We will solve this in the same way we dealt with the previous dataset: group by id and get the mean. 

In [10]:
df_1 = df_1.groupby('id').mean().reset_index()

# Checking that we have no duplicates in id column
print(df_1.id.duplicated().sum())

# Checking that the lenght of the dataset is equal to the unique instances of the dataset. 
df_1.id.nunique() == len(df_1)

0


True

## Checking DF_2

In [11]:
general_info(df_2)

,type,count,na,na%
id,object,100000,0,0.0
f0,float64,100000,0,0.0
f1,float64,100000,0,0.0
f2,float64,100000,0,0.0
product,float64,100000,0,0.0



Sample:


,id,f0,f1,f2,product
5218,EYuGF,1.980388,1.044908,1.300817,58.666450
47879,8etwE,1.574434,4.983617,3.425385,128.787131
18129,OLL6u,2.130182,0.248854,1.722685,39.740780
31400,CePFp,1.430780,-1.614388,-1.320081,173.754635
45023,2rmtK,-0.591297,0.212175,3.641514,54.477672



Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

...

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838



Nulls in the columns:


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Zeros in the columns:

Shape: (100000, 5)

Unique wells in the dataset : 99996

Duplicated: We have 0 duplicated rows


Dtypes:


id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

In [12]:
# Checking the number of duplicated ids
df_2.id.duplicated().sum()

4

## Fixing df_2:

df_2 dataset contains 4 duplicates in the id column. Let's group by id and avr.

In [13]:
df_2 = df_2.groupby('id').mean().reset_index()

# Checking that we have no duplicates in id column
print(df_2.id.duplicated().sum())

# Checking that the lenght of the dataset is equal to the unique instances of the dataset. 
df_2.id.nunique() == len(df_2)

0


True

<div class="alert alert-success">
<b>Reviewer's comment</b>

Alright, the data was loaded and inspected

</div>

### So far we imported the datasets, got familiar with the numbers, noticed the anomalies (8K instances of product == 0 in df_1), and got rid of the duplicates. 

**Our next step would be :**

* To split the data into a training set and validation set at a ratio of 75:25.
* To train the model and make predictions for the validation set.
* To save the predictions and correct answers for the validation set.
* To print the average volume of predicted reserves and model RMSE.
* To analyze the results.

In [14]:
## Let's do the following:

In [15]:
df_0.name = "df_0"
df_1.name = "df_1"
df_2.name = "df_2"


for data in [df_0,df_1, df_2]:
    # Let's split our features and target to test and train subsets:
    features = data[["f0", "f1",  "f2"]]
    target  = data["product"]  
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)
    
    #Let's instantiate the model:
    model = LinearRegression()

    # Training the model:
    model.fit(X_train, y_train)
    
    # Predicting the values of the test features and saving in the predicted values variable:
    predicted_values = model.predict(X_test)
    
    # Calculating the Root_mean_squared_error (RMSE) 
    rmse = mean_squared_error(y_test, predicted_values, squared=False)
    print("RMSE of", data.name, "is",round(rmse, 2), ", avr volume of predicted reserves:", round(predicted_values.mean(),2))

RMSE of df_0 is 37.62 , avr volume of predicted reserves: 92.46
RMSE of df_1 is 0.89 , avr volume of predicted reserves: 68.85
RMSE of df_2 is 39.86 , avr volume of predicted reserves: 94.92


## Analyzing the results:

We performed 3 tests of different models, each referring to a different geographical area. 
There's not likely to be any acceptable "good" value for RMSE. These scores are better interpreted and applied comparatively rather than absolutely. Yet as a rule of a tumb, in terms of RMSE, the lower the better. Our best performing model so far is the second one (of df_1), with the lowest RMSE of 0.88. Our worst model is the third one (df_2) with RMSE of 39.8

<div class="alert alert-success">
<b>Reviewer's comment</b>

The models were trained and evaluated correctly

</div>

Yet our suspicion is that the unusually good RMSE score of region df_1 is due to the high volume of 
product = 0 instances. 
Let's make a test and run this check again, this time without product = 0 instances:

In [16]:
testing_subset = df_1[df_1["product"]>0]
print("The lenght of testing subset of df_1 region is", len(testing_subset), "instances")
print()

testing_subset.name = "testing_subset"

for data in [df_0,df_1, df_2, testing_subset]:
    # Let's split our features and target to test and train subsets:
    features = data[["f0", "f1",  "f2"]]
    target  = data["product"]  
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)
    
    #Let's instantiate the model:
    model = LinearRegression()

    # Training the model:
    model.fit(X_train, y_train)
    
    # Predicting the values of the test features and saving in the predicted values variable:
    predicted_values = model.predict(X_test)
    
    # Calculating the Root_mean_squared_error (RMSE) 
    rmse = mean_squared_error(y_test, predicted_values, squared=False)
    print("RMSE of", data.name, "is", round(rmse, 2), ", avr volume of predicted reserves:", round(predicted_values.mean(),2))

The lenght of testing subset of df_1 region is 91761 instances

RMSE of df_0 is 37.62 , avr volume of predicted reserves: 92.46
RMSE of df_1 is 0.89 , avr volume of predicted reserves: 68.85
RMSE of df_2 is 39.86 , avr volume of predicted reserves: 94.92
RMSE of testing_subset is 0.89 , avr volume of predicted reserves: 74.57


Seems like the good results of df_1 are NOT due to the high number of product=0 instances.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Indeed

</div>

# 3. Profit Calculations

### 3.1 Storing all key values for calculations in separate variables.


In [17]:

cost_of_200_wells = 100000000 # (one hundred million)
number_of_wells = 200
cost_of_one_well = cost_of_200_wells / number_of_wells
unit_of_product = 4500
cost_of_one_well

500000.0

### 3.2 Calculate the volume of reserves sufficient for developing a new well without losses. Compare the obtained value with the average volume of reserves in each region.

In [18]:
volume_of_reserves_sufficient = cost_of_one_well / unit_of_product 
volume_of_reserves_sufficient

print()
print("The volume of reserves sufficient for developing a new well without losses is", round(volume_of_reserves_sufficient, 2), "units")
print("and yet our mean of products in the regions are:")
print()

for region in [df_0, df_1, df_2]:
    print("Mean volume of reserves of", region.name, " region is", round(region["product"].mean(),2))


The volume of reserves sufficient for developing a new well without losses is 111.11 units
and yet our mean of products in the regions are:

Mean volume of reserves of df_0  region is 92.5
Mean volume of reserves of df_1  region is 68.82
Mean volume of reserves of df_2  region is 95.0


### 3.3. Provide the findings about the preparation for profit calculation step.

So far we have found that the region with the most precise prediction product estimates its mean product to be 68.8 units, the lowest value of the three. The other regions are in the lower 90 of mean units. Which is not sufficient for covering the costs of the well development. 
Meaning that the company involved in the well development takes a risk.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Calculations are correct!

</div>

### 4.1 - 4.2  Pick the wells with the highest values of predictions. Summarize the target volume of reserves in accordance with these predictions

In [19]:
def profit(list_to_inspect):
    print("True profit of 200 wells with best predictions:")
    print(" ")
    price_of_unit = 4500
    cost_of_200_wells = 100000000
    for data in list_to_inspect:
        # Let's split our features and target to test and train subsets:
        features = data[["f0", "f1",  "f2"]]
        target  = data["product"]  
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

        #Let's instantiate the model:
        model = LinearRegression()

        # Training the model:
        model.fit(X_train, y_train)

        # Predicting the values of the test features and saving in the predicted values variable:
        predicted_values = pd.Series(model.predict(X_test))
        
        best_200_predictions = predicted_values.nlargest(200)
        best_picks = [y_test.iloc[i] for i in best_200_predictions.index]
        target_volume = sum(best_picks)
        print(f"Region: {data.name} /  Revenue: {round(target_volume*price_of_unit/1000000, 2)} million USD. Profit: {round((target_volume*price_of_unit-cost_of_200_wells)/1000000, 2)} ")
        #print(f"Region:  /  Revenue: {round(target_volume*price_of_unit/1000000, 2)} mill USD. Profit: {round((target_volume*price_of_unit-cost_of_200_wells)/1000000, 2)} mill USD")


In [20]:
regions = [df_0,df_1, df_2]
profit(regions)

True profit of 200 wells with best predictions:
 
Region: df_0 /  Revenue: 133.72 million USD. Profit: 33.72 
Region: df_1 /  Revenue: 124.15 million USD. Profit: 24.15 
Region: df_2 /  Revenue: 123.65 million USD. Profit: 23.65 


### 4.3  Provide findings: suggest a region for oil wells' development and justify the choice. 

It would be logical to suggest that df_0 is the best performing region and therefore is the leading condidate for well development. Yet we must take into the account the different risk levels and the different levels of predictability of our model (RMSE)

In order to suggest a region for oil wells development we must take into account the risk involved in each region, the predictability of the profits. 



<div class="alert alert-success">
<b>Reviewer's comment V2</b>

Alright, looks good!

</div>

### 5.1 - 5.2  Use the bootstrapping technique with 1000 samples to find the distribution of profit. Find average profit, 95% confidence interval and risk of losses. Loss is negative profit, calculate it as a probability and then express as a percentage.

This task was quite challenging as it was ambigious and there were many many wrong interpretations made by me and other students. From my best understanding of the task, following your comments, I used this logic:

- We devide our 3 regions by 1000 samples each,
- Each sample contains 500 instances of wells. 
- We run a model on each sample and get the predictions 
- We sort these predictions and use the best 200 predictions' indexes to choose 200 true products of the wells.
- Thus we can calculate the avr revenue, profit and risk. 



In [26]:
def bootstraping(data_list):

    for data in data_list:
        cost_of_200_wells = 100000000
        features = data[["f0", "f1",  "f2"]]
        target  = data["product"]        
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42) 
                # Let's instantiate the model:
        model = LinearRegression()     
                # Training the model:
        model.fit(X_train, y_train)       

        list_of_prods = []
        list_of_revs = []
        list_of_profits = []
        for i in range(1000):
                subsample = data.sample(n=500,  replace=True)

                # Predicting the values of the test features and saving in the predicted values variable:
                predicted_values = pd.Series(model.predict(subsample[["f0", "f1",  "f2"]])) 
                # Selecting 200 best predictions
                best_200_predictions = predicted_values.nlargest(200)  
                # Using the indexes of best preds to find the wells 
                best_prods = [subsample["product"].iloc[ind] for ind in best_200_predictions.index]
                list_of_prods.append(sum(best_prods))
                list_of_revs.append(sum(best_prods)*4500)
                list_of_profits.append((sum(best_prods)*4500) - cost_of_200_wells)

        avr_true_product = np.mean(list_of_prods)
        avr_rev = avr_true_product*4500
        avr_profit = avr_rev - cost_of_200_wells
        lower = pd.Series(list_of_profits).quantile(0.025)
        upper = pd.Series(list_of_profits).quantile(0.975)
        neg_count = len(list(filter(lambda x: (x < 0), list_of_profits)))
                


        print(f"    Region: {data.name} \n" 
                  f"Avr Revenue {round(avr_rev,2)}  USD \n" 
                  f"Avr Profit {round(avr_profit,2)}  USD \n" 
                  f"Lower Profit {round(lower,2)} USD\n" 
                  f"Upper Profit {round(upper,2)}  USD\n" 
                  f"Risk of Losses {round(neg_count/len(list_of_profits)*100,2)}% \n"
                  "################# \n"
                   " ")
regions = [df_0,df_1, df_2]
bootstraping(regions)

    Region: df_0 
Avr Revenue 104232732.99  USD 
Avr Profit 4232732.99  USD 
Lower Profit -570194.17 USD
Upper Profit 9133042.35  USD
Risk of Losses 4.1% 
################# 
 
    Region: df_1 
Avr Revenue 104533359.92  USD 
Avr Profit 4533359.92  USD 
Lower Profit 510389.78 USD
Upper Profit 8431289.73  USD
Risk of Losses 1.0% 
################# 
 
    Region: df_2 
Avr Revenue 103746295.39  USD 
Avr Profit 3746295.39  USD 
Lower Profit -1546313.08 USD
Upper Profit 8931852.11  USD
Risk of Losses 9.0% 
################# 
 


<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Great, everything is correct now!

</div>

## Thank you, these were very important points!

### Conclution:

Our analysis shows that **region df_1**, is the best performing region in our task.

Having the **highiest mean profit** of the three regions, the df_1 region also benefits from:
* The **lowest risk of losses**, 
* **Highest lower confidence** level. 
* **The Lowest RMSE**, meaning that the predictability of the product estimates in this region are very high (thus, the risk is low)

<div class="alert alert-success">
<b>Reviewer's comment V3</b>

Region choice is correct and justified!

</div>